In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import umap

from postprocessor.core.processes.catch22 import catch22
from postprocessor.core.processes.standardscaler import standardscaler

In [ ]:
from src.umapper import umapper

# Load data

In [ ]:
data_dir = "../data/raw/"
group1_name = "st01253_tsa1tsa2morgan"
group2_name = "st01253_by4742swain"

In [ ]:
filepath1 = data_dir + group1_name
timeseries1_filepath = filepath1 + "_timeseries.csv"
labels1_filepath = filepath1 + "_labels.csv"

timeseries1_df = pd.read_csv(timeseries1_filepath, index_col=[0,1,2])
labels1_df = pd.read_csv(labels1_filepath, index_col=[0,1,2])


filepath2 = data_dir + group2_name
timeseries2_filepath = filepath2 + "_timeseries.csv"
labels2_filepath = filepath2 + "_labels.csv"

timeseries2_df = pd.read_csv(timeseries2_filepath, index_col=[0,1,2])
labels2_df = pd.read_csv(labels2_filepath, index_col=[0,1,2])

Join dataframes

In [ ]:
timeseries_df = pd.concat([timeseries1_df, timeseries2_df])
labels_df = pd.concat([labels1_df, labels2_df])

In [ ]:
timeseries_df

In [ ]:
labels_df

# Featurise (time series --> catch22)

In [ ]:
features_df = catch22.as_function(timeseries_df)

In [ ]:
features_df

# Scale (standardscaler)

In [ ]:
features_scaled = standardscaler.as_function(features_df.T).T

In [ ]:
features_scaled

# UMAP

## Old way

In [ ]:
# Bodge: add strain so it plays well with old umapper code
features_strain_df = pd.concat({'tsa1tsa2morgan': features_df}, names=['strain'])

In [ ]:
fig_umap, ax_umap = plt.subplots(figsize=(6,6))
umapper.umap_plot(
    data=features_strain_df,
    n_neighbors=20,
    min_dist=0.5,
    n_components=2,
    label_index="strain",
    ax=ax_umap
)

## Refactor: broken down, for more flexibility

In [ ]:
reducer = umap.UMAP(
    n_neighbors=10,
    min_dist=0.05,
    n_components=2,
)

In [ ]:
mapper = reducer.fit(features_scaled)
embedding = mapper.embedding_

Dummy colour scheme

In [ ]:
# Dummy colour scheme
test_list = ['foo']*300 + ['bar']*52
test_palette_map = {
    'foo': 'r',
    'bar': 'k',
}

fig, ax = plt.subplots(figsize=(6,6))
sns.scatterplot(
    x=embedding[:,0],
    y=embedding[:,1],
    hue=test_list,
    palette=test_palette_map,
    ax=ax,
)

Colour dots by strain

In [ ]:
position_list = features_scaled.index.get_level_values("position").to_list()
strain_list = [position.split("_")[0] for position in position_list]
strain_palette_map = {
    'tsa1tsa2morgan': 'r',
    'by4741': 'k',
}

fig, ax = plt.subplots(figsize=(6,6))
sns.scatterplot(
    x=embedding[:,0],
    y=embedding[:,1],
    hue=strain_list,
    palette=strain_palette_map,
    ax=ax,
)

Colour dots by score

In [ ]:
common_idx = features_scaled.index.intersection(labels_df.index)
scores_list = labels_df.loc[common_idx].score.to_list()
scores_relabel_lookup = {
    0: 'Oscillatory',
    1: 'Non-oscillatory',
}
scores_list = [scores_relabel_lookup.get(item,item) for item in scores_list]
scores_palette_map = {
    'Oscillatory': 'r',
    'Non-oscillatory': 'k',
}

fig, ax = plt.subplots(figsize=(6,6))
sns.scatterplot(
    x=embedding[:,0],
    y=embedding[:,1],
    hue=scores_list,
    palette=scores_palette_map,
    ax=ax,
)